# Extract and Transform Weather Data

#### In this notebook we will be filtering a bulk weather csv file downloaded from OpenWeatherAPI's History Bulk purchase option and transform desired information in order to build the dataset into our sqlite database.

1. Data source: [OpenWeather History Bulk](https://openweathermap.org/history-bulk)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the csv into a df
file_path = '../resources/raw_data/raw_historical_bulk_weather_philly.csv'

weather_df = pd.read_csv(file_path)
weather_df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1420070400,2015-01-01 00:00:00 +0000 UTC,-18000,Philadelphia,39.952584,-75.165222,29.19,17.55,26.74,30.92,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,1420074000,2015-01-01 01:00:00 +0000 UTC,-18000,Philadelphia,39.952584,-75.165222,28.72,18.52,26.25,30.92,...,200,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
2,1420077600,2015-01-01 02:00:00 +0000 UTC,-18000,Philadelphia,39.952584,-75.165222,28.85,18.77,26.02,30.20,...,190,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,1420081200,2015-01-01 03:00:00 +0000 UTC,-18000,Philadelphia,39.952584,-75.165222,28.31,14.95,25.33,30.02,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,1420084800,2015-01-01 04:00:00 +0000 UTC,-18000,Philadelphia,39.952584,-75.165222,27.28,14.54,24.82,28.94,...,230,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [3]:
# Check columns
weather_df.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [4]:
# Select only desired columns
weather_df = weather_df [['dt', 'dt_iso', 'city_name', 'lat', 'lon',
                          'temp', 'feels_like', 'temp_min', 'temp_max', 'humidity',
                          'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_1h',
                           'snow_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description',
                           'weather_icon'
                         ]]
weather_df.head()

,dt,dt_iso,city_name,lat,lon,temp,feels_like,temp_min,temp_max,humidity,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1420070400,2015-01-01 00:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,29.19,17.55,26.74,30.92,42,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,1420074000,2015-01-01 01:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.72,18.52,26.25,30.92,48,...,200,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
2,1420077600,2015-01-01 02:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.85,18.77,26.02,30.20,52,...,190,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,1420081200,2015-01-01 03:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.31,14.95,25.33,30.02,53,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,1420084800,2015-01-01 04:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,27.28,14.54,24.82,28.94,55,...,230,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [5]:
# Check row count
len(weather_df)

58344

##### Downloaded data is from 2015-2021, we are only interested in the 2017 data. Filter by the datetime field.

In [6]:
# To do this, first we need to parse the UTC dt format to a more readable format
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit='s')
weather_df.head()

,dt,dt_iso,city_name,lat,lon,temp,feels_like,temp_min,temp_max,humidity,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00,2015-01-01 00:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,29.19,17.55,26.74,30.92,42,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,2015-01-01 01:00:00,2015-01-01 01:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.72,18.52,26.25,30.92,48,...,200,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
2,2015-01-01 02:00:00,2015-01-01 02:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.85,18.77,26.02,30.20,52,...,190,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,2015-01-01 03:00:00,2015-01-01 03:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,28.31,14.95,25.33,30.02,53,...,220,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,2015-01-01 04:00:00,2015-01-01 04:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,27.28,14.54,24.82,28.94,55,...,230,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [7]:
# Filter for data from 2017
filtered_df_2017 = weather_df[weather_df['dt'].dt.year == 2017]
filtered_df_2017.head()

,dt,dt_iso,city_name,lat,lon,temp,feels_like,temp_min,temp_max,humidity,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
18261,2017-01-01 00:00:00,2017-01-01 00:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,...,200,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
18262,2017-01-01 01:00:00,2017-01-01 01:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,...,210,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
18263,2017-01-01 02:00:00,2017-01-01 02:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,43.79,30.16,41.0,46.70,46,...,210,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
18264,2017-01-01 03:00:00,2017-01-01 03:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,44.01,30.34,41.0,46.58,45,...,220,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
18265,2017-01-01 04:00:00,2017-01-01 04:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,45.27,30.87,42.8,46.40,41,...,220,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n


In [8]:
# Reset index
filtered_df_2017.reset_index(inplace=True, drop=True)

In [9]:
# Check row count and NaN count
print(f'# of rows: {len(filtered_df_2017)}')

filtered_df_2017.isna().sum()

# of rows: 9125


dt                        0
dt_iso                    0
city_name                 0
lat                       0
lon                       0
temp                      0
feels_like                0
temp_min                  0
temp_max                  0
humidity                  0
wind_speed                0
wind_deg                  0
rain_1h                8072
rain_3h                8921
snow_1h                8977
snow_3h                9105
clouds_all                0
weather_id                0
weather_main              0
weather_description       0
weather_icon              0
dtype: int64

##### As expected, a large percentage of the year were not rainy days and did not snow. However, instead of removing the NaN, we will fill them to have a value of numeric 0. Our assumption is having a value instead of NaN in those rows will be beneficial further down the road in our analysis and visualizations

In [10]:
# Fill all NaN with 0
filled_df = filtered_df_2017.fillna(0)
# Make sure all rows have values populated
filled_df.isna().any()

dt                     False
dt_iso                 False
city_name              False
lat                    False
lon                    False
temp                   False
feels_like             False
temp_min               False
temp_max               False
humidity               False
wind_speed             False
wind_deg               False
rain_1h                False
rain_3h                False
snow_1h                False
snow_3h                False
clouds_all             False
weather_id             False
weather_main           False
weather_description    False
weather_icon           False
dtype: bool

In [11]:
# Check datatypes
filled_df.dtypes

dt                     datetime64[ns]
dt_iso                         object
city_name                      object
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [12]:
# Cast weather_id to a string since it's not a continous integer and it only serves as a code for weather description
filled_df['weather_id'] = filled_df['weather_id'].astype(str)

In [13]:
filled_df.head(2)

,dt,dt_iso,city_name,lat,lon,temp,feels_like,temp_min,temp_max,humidity,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2017-01-01 00:00:00,2017-01-01 00:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,...,200,0.0,0.0,0.0,0.0,90,804,Clouds,overcast clouds,04n
1,2017-01-01 01:00:00,2017-01-01 01:00:00 +0000 UTC,Philadelphia,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,...,210,0.0,0.0,0.0,0.0,75,803,Clouds,broken clouds,04n


##### Datatypes are all desired types. Now we will just further filter out some columns that we are unlikely to use in our project

In [14]:
# Remove the other datetime field "dt_iso" since it's duplicated and "city_name" since all data is for Philadelphia
cleaned_df = filled_df.drop(columns=['dt_iso', 'city_name'])
cleaned_df.head(2)

,dt,lat,lon,temp,feels_like,temp_min,temp_max,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2017-01-01 00:00:00,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,18.34,200,0.0,0.0,0.0,0.0,90,804,Clouds,overcast clouds,04n
1,2017-01-01 01:00:00,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,13.87,210,0.0,0.0,0.0,0.0,75,803,Clouds,broken clouds,04n


In [15]:
# Print datatypes one more time for reference
cleaned_df.dtypes

dt                     datetime64[ns]
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                     object
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

##### One more thing that we have to check before we export... Since the number of rows seems to exceed the number of hours in a year (365*24=8760), there might be some duplicated rows.

In [16]:
# The only field we have to make sure it's unique is the datetime column. Check if there are duplicated rows
cleaned_df["dt"].duplicated().any()

True

In [17]:
# Since there are, we want to see which ones and explore more
cleaned_df[cleaned_df["dt"].duplicated()].head(10)

,dt,lat,lon,temp,feels_like,temp_min,temp_max,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
35,2017-01-02 10:00:00,39.952584,-75.165222,35.96,30.31,32.0,39.64,81,3.36,20,0.75,0.0,0.3,0.0,90,600,Snow,light snow,13n
37,2017-01-02 11:00:00,39.952584,-75.165222,35.65,29.37,32.0,40.11,85,4.70,30,1.40,0.0,0.8,0.0,90,601,Snow,snow,13n
39,2017-01-02 12:00:00,39.952584,-75.165222,36.21,28.76,32.0,40.26,85,6.93,80,0.80,0.0,0.3,0.0,90,600,Snow,light snow,13n
43,2017-01-02 15:00:00,39.952584,-75.165222,38.75,31.05,35.6,42.25,85,8.05,80,0.30,1.4,0.0,0.0,90,701,Mist,mist,50d
45,2017-01-02 16:00:00,39.952584,-75.165222,39.99,32.63,35.6,43.17,88,8.05,70,0.30,0.0,0.0,0.0,90,701,Mist,mist,50d
47,2017-01-02 17:00:00,39.952584,-75.165222,40.19,33.69,35.6,43.43,92,6.93,50,0.60,0.0,0.0,0.0,90,701,Mist,mist,50d
50,2017-01-02 19:00:00,39.952584,-75.165222,41.45,32.67,37.4,43.74,92,11.41,40,0.87,0.0,0.0,0.0,90,701,Mist,mist,50d
54,2017-01-02 22:00:00,39.952584,-75.165222,41.13,32.86,37.4,43.33,91,10.29,60,0.30,0.0,0.0,0.0,90,500,Rain,light rain,10n
55,2017-01-02 22:00:00,39.952584,-75.165222,41.13,32.86,37.4,43.33,91,10.29,60,0.30,0.0,0.0,0.0,90,701,Mist,mist,50n
62,2017-01-03 04:00:00,39.952584,-75.165222,40.75,30.45,37.4,44.74,92,13.87,40,0.00,0.0,0.0,0.0,90,701,Mist,mist,50n


In [18]:
# Check number of dt rows that are duplicated
print(f'Duplicated dt rows: {len(cleaned_df[cleaned_df["dt"].duplicated()])}')
print(f'2017 rows: {len(cleaned_df)}')

Duplicated dt rows: 365
2017 rows: 9125


##### The number of rows in the filtered dataset was 9125 and the duplicated rows we just found was 365. 9125-365=8760 which is the exact number of hours a typical year should have.

In [19]:
# Now we will drop those duplicates.

# We do not have enough insights as to why there are 365 duplicated hours, but the data from the duplicated rows
# don't look particularly different so in this case since the number of rows is small we will just default to a 
# (keep='first') method while using pd.drop_duplicates()
final_df = cleaned_df.drop_duplicates(subset='dt', keep='first')

In [20]:
# Check number of rows to make sure everything's done right
len(final_df)

8760

##### Add a few datetime manipulated columns for later convenience...

In [22]:
# Add a day column
final_df['month'] = final_df['dt'].dt.month
# Add an day column
final_df['day'] = final_df['dt'].dt.day
# Add an hour column
final_df['hour'] = final_df['dt'].dt.hour

/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [23]:
# New datetime column used for parsing. Will be deleted later.
final_df['dt_2'] = final_df['dt']
final_df['dt_2'] = final_df['dt_2'].astype(str)

/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# Add a year-month-day-hour identifier column. This will come in handy for our our analysis such as scatter plot
ymdh = []

for datetime in final_df['dt_2']:
    ymdh.append(datetime.split(':')[0])
    
final_df['ymdh'] = ymdh

/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [25]:
final_df = final_df.drop(columns='dt_2')

In [26]:
final_df

,dt,lat,lon,temp,feels_like,temp_min,temp_max,humidity,wind_speed,wind_deg,...,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,month,day,hour,ymdh
0,2017-01-01 00:00:00,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,18.34,200,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,0,2017-01-01 00
1,2017-01-01 01:00:00,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,13.87,210,...,0.0,75,803,Clouds,broken clouds,04n,1,1,1,2017-01-01 01
2,2017-01-01 02:00:00,39.952584,-75.165222,43.79,30.16,41.0,46.70,46,16.11,210,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,2,2017-01-01 02
3,2017-01-01 03:00:00,39.952584,-75.165222,44.01,30.34,41.0,46.58,45,16.11,220,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,3,2017-01-01 03
4,2017-01-01 04:00:00,39.952584,-75.165222,45.27,30.87,42.8,46.40,41,17.22,220,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,4,2017-01-01 04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,2017-12-31 19:00:00,39.952584,-75.165222,15.62,0.07,14.0,18.11,39,16.11,310,...,0.0,90,804,Clouds,overcast clouds,04d,12,31,19,2017-12-31 19
9121,2017-12-31 20:00:00,39.952584,-75.165222,15.51,0.57,14.0,17.07,39,14.99,320,...,0.0,90,804,Clouds,overcast clouds,04d,12,31,20,2017-12-31 20
9122,2017-12-31 21:00:00,39.952584,-75.165222,14.85,-1.30,12.2,16.63,43,17.22,310,...,0.0,90,804,Clouds,overcast clouds,04d,12,31,21,2017-12-31 21
9123,2017-12-31 22:00:00,39.952584,-75.165222,13.62,-1.89,12.2,15.17,45,16.11,320,...,0.0,40,802,Clouds,scattered clouds,03n,12,31,22,2017-12-31 22


In [27]:
final_df.dtypes

dt                     datetime64[ns]
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                     object
weather_main                   object
weather_description            object
weather_icon                   object
month                           int64
day                             int64
hour                            int64
ymdh                           object
dtype: object

#### Now we feel comfortable enough to export the dataset

In [28]:
# Export to CSV
final_df.to_csv('../resources/cleaned_data/cleaned_philly_weather_2017.csv', index=False)

Creating some data facts and references for later use...

In [29]:
# # Generating a min-max value dataframe for numeric values only
# columns = list(final_df.columns)
# max = []
# min = []

# for column in columns:
#     min.append(final_df.loc[:, f'{column}'].min())
#     max.append(final_df.loc[:, f'{column}'].max())

# min_max_df = pd.DataFrame(
#     data = {
#         'min': min,
#         'max': max
#     },
#     index = columns
# )

# min_max_df.iloc[-4:, :] = 'String'

# min_max_df

In [30]:
# min_max_df